In [ ]:
%load_ext autoreload
%autoreload 2
#%matplotlib notebook
# produce V1_allpos, V1_feature, V1_RF

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clr
from scipy import integrate
from patch_square import *
#fdr = 'D:/scratch/patchV1/resource/'
#fdr = 'C:/Users/gueux/Repos/patchV1/Data/'
fdr = '/root/autodl-tmp/public/resource/'
seed = 6532787
rng = np.random.default_rng(seed = seed)

In [ ]:
ecc0 = 2
deg2_per_mm2 = ecc0*ecc0*np.pi/2/(217.364/2) # halved for single eye condition (ocular dominance columns for ipsi and contra are mostly balanced)
print(deg2_per_mm2)
# 2.0 ecc yields 217.364 𝑚𝑚2 ~ 0.0289 deg2 per mm2(O'Kusky & Colonnier 1982)
# L2/3 model density ~ 14766.156 per 𝑚𝑚2 (18)
# L4Cbeta model density ~ 24613.409 per 𝑚𝑚2 (30)
# L4Calpha model density ~ 14193.732 per mm2 (17.3)

#    4Cbeta  4Calpha L2/3
# #   480      272    272 = 1024
# E/I 4:1      3:1    3:1  
#              E              I
# block: 384, 204, 204 | 96, 68, 68

# using uniform density
theme = 'square-single_layer'
#den_L2_3 = 46020.984
den_L4beta = 24613.409/4
#den_L4alpha = 14193.732
#den = np.array([den_L2_3, den_L4beta, den_L4alpha])
den = np.array([den_L4beta])
nLayer = 1
nV1 = np.zeros(nLayer, dtype = 'u4')
nblock = np.zeros(nLayer, dtype = 'u4')
blockSize = np.zeros(nLayer, dtype = 'u4')
grid_1D = np.zeros(nLayer, dtype = 'u4')
theme = '_'+theme

deg = 1
ecc = deg/2
warpSize = 32
d0 = 0.5
area = deg*deg/deg2_per_mm2
print(area)
nOP_x = int(np.sqrt(area)/d0)
nOP_y = nOP_x
nOP = nOP_x * nOP_y
d = np.sqrt(area)/nOP_x
r = d/2
w = d*nOP_x
h = d*nOP_y
print(f'diameter of nOP = {d:.4}mm')

In [ ]:
for i in range(nLayer):
    nV1[i] = int(round(den[i]*area/warpSize))*warpSize
    nV1_32 = nV1[i]//warpSize
    min_residual = np.inf
    for j in range(2,int(np.sqrt(nV1_32))):
        nWarpPerBlock = round(nV1_32/(j*j))
        residual = abs(nV1_32/(j*j) - nWarpPerBlock)*j*j
        if nWarpPerBlock <= 32 and residual < min_residual:
            min_residual = residual
            nblock[i] = j*j
            grid_1D[i] = j
            blockSize[i] = nWarpPerBlock*warpSize
print(f'putative nV1 = {nV1}')
nV1 = nblock*blockSize
print('suggested:')
for i in range(nLayer):
    print(f'{grid_1D[i]} x {grid_1D[i]} x {blockSize[i]} = {nV1[i]}')

In [ ]:
#blockSize = np.array([832,448,256])
#grid_1D = np.array([31,31,31])
blockSize = np.array([1024])
grid_1D = np.array([10])
nblock = grid_1D*grid_1D
nV1 = nblock*blockSize
nE = 876
print(nV1)
area = np.mean(nV1/den)
#per_dis = np.sqrt(area)/grid_1D
per_dis = w/grid_1D
vis = np.sqrt(deg2_per_mm2*w*h)
print(f'{area} mm2 ~ {vis:.3f} x {vis:.3f} deg2')
print(f'{nOP_x}x{nOP_y} hypercolumns, d={d:.3f}mm')
for i in range(nLayer):
    blocksPerHypercolumn = (d/per_dis[i])*(d/per_dis[i])
    print(f'{grid_1D[i]}x{grid_1D[i]}:{blockSize[i]} = {nV1[i]} neurons, {blocksPerHypercolumn:.2f} blocks per column')

In [ ]:
pos_filename = fdr + 'V1_allpos'+theme+'.bin'
fig = plt.figure(figsize = [10, 10], dpi = 300)
ax = fig.add_subplot(111)
pos = np.empty(nLayer, dtype = object)
block_center = np.empty(nLayer, dtype = object)
with open(pos_filename, 'wb') as f:
    #np.array([nLayer]).astype('u4').tofile(f)
    nblock.astype('u4').tofile(f)
    blockSize.astype('u4').tofile(f)
    for i in range(nLayer):
        pos_range = np.array([-grid_1D[i]/2, grid_1D[i]/2])*per_dis[0]
        #print(pos_range)
        pos[i] = np.zeros((nblock[i], 2, blockSize[i]), dtype = 'f8')
        block_center[i] = np.zeros((nblock[i],2))
        block_center[i][:,0] = np.tile((np.arange(grid_1D[i])-grid_1D[i]/2 + 1/2)*per_dis[i], grid_1D[i])
        block_center[i][:,1] = np.repeat((np.arange(grid_1D[i])-grid_1D[i]/2 + 1/2)*per_dis[i], grid_1D[i])
        fpos = np.zeros((2,nblock[i]*blockSize[i]), dtype = 'f8')
        for j in range(nblock[i]):
            pos[i][j,:,:] = square_pos(per_dis[i], blockSize[i], block_center[i][j,:], rng = rng)
            fpos[0,j*blockSize[i]:(j+1)*blockSize[i]] = pos[i][j,0,:]
            fpos[1,j*blockSize[i]:(j+1)*blockSize[i]] = pos[i][j,1,:]
            ax.plot(pos[i][j,0,:nE], pos[i][j,1,:nE], ',')
            ax.plot(pos[i][j,0,nE:], pos[i][j,1,nE:], 's', ms = 0.05)
        ax.plot(block_center[i][:,0], block_center[i][:,1], '*k', ms = 0.1)
        for j in range(nblock[i]):
            ax.text(block_center[i][j,0]-0.05, block_center[i][j,1]+0.025, f'{j}', fontsize='xx-small', color = 'k')
            ax.text(block_center[i][j,0]-0.075, block_center[i][j,1]-0.04, f'{block_center[i][j,0]:.2f}', fontsize=3.5, color = 'k')
            ax.text(block_center[i][j,0]-0.075, block_center[i][j,1]-0.075, f'{block_center[i][j,1]:.2f}', fontsize=3.5, color = 'k')
        fpos.tofile(f)
        vpos = (fpos - pos_range[0])/(pos_range[1]-pos_range[0])*2*ecc-ecc
        vpos.astype('f8').tofile(f)
ax.set_aspect('equal')

In [ ]:
# old
pos_filename = fdr + 'V1_allpos'+theme+'.bin'
with open(pos_filename, 'wb') as f:
    nblock.astype('u4').tofile(f)
    blockSize.astype('u4').tofile(f)
    np.array([2]).astype('u4').tofile(f)
    span = np.array([pos_range[0], pos_range[1]-pos_range[0], pos_range[0], pos_range[1]-pos_range[0]])
    span.tofile(f)
    fpos.tofile(f)
    vspan = np.array([-ecc, 2*ecc, -ecc, 2*ecc])
    vspan.tofile(f)
    vpos.tofile(f)

In [ ]:
# old
vpos_filename = fdr + 'V1_vpos'+theme+'.bin'
n = vpos.shape[1]
V1_vpos = np.empty((2,n),dtype ='f8')
V1_vpos[0,:] = np.sqrt(vpos[0,:]*vpos[0,:] + vpos[1,:]*vpos[1,:])
V1_vpos[1,:] = np.arctan2(vpos[1,:], vpos[0,:])
with open(vpos_filename, 'wb') as f:
    np.array([n]).astype('u4').tofile(f)
    V1_vpos.tofile(f)

In [ ]:
def get_condition(i,j,k):
    condition = j == nOP_y - 1 and k == 7 #and (k == 0 or k == 2)
    #return condition
    return True

In [ ]:
# features: OP, OD
fp = 'f4'
plot = True 
check = False
OP_center_y = (np.arange(nOP_y)-nOP_y/2+1/2)*d
phase0 = 0
rot_rate = 16.3702215*(1+3/4)/180*np.pi
if plot:
    fig = plt.figure(figsize = (8, 8), dpi = 600)
    marker = ('s', '^', 'o')
OP = np.empty(nLayer, dtype = object)
LR = np.empty(nLayer, dtype = object)
guides = []
if check:
    check_fig = plt.figure()
    OP_row = []
for i in range(nLayer):
    #ax = fig.add_subplot(1,3,i+1)
    ax = fig.add_subplot(1,1,1)
    OP[i] = np.zeros((nblock[i],blockSize[i]), dtype = 'f4')
    LR[i] = np.zeros((nblock[i],blockSize[i]), dtype = 'f4')-1  # single eye
    block_r = per_dis[i] * np.sqrt(2)
    for j in range(nOP_y):
        OP_center = np.zeros((2,nOP_x))
        OP_center[0,:] = (np.arange(nOP_x)-nOP_x/2+1/2)*d
        OP_center[1,:] = OP_center_y[j]
        val0 = 1.0
        sat0 = 1.0
        for k in range(nOP_x):
            center = OP_center[:,k]
            phase_sign = (np.mod(j,2)-0.5)*2
            #print(rot_rate*k/nOP_x*180)
            guide_x = [center[0]]
            guide_y = [center[1]]
            if np.mod(k,2) == 0:
                phase = 0 + phase_sign*rot_rate*k/nOP_x*np.pi
                guide_phase = 0 + phase_sign*(rot_rate*k/nOP_x*np.pi - np.pi*3/4)
            else:
                phase = np.pi - phase_sign*rot_rate*k/nOP_x*np.pi
                guide_phase = np.pi - phase_sign*(rot_rate*k/nOP_x*np.pi - np.pi*3/4)
                    
            guide_x.append(r * np.cos(guide_phase) + center[0])
            guide_y.append(r * np.sin(-guide_phase) + center[1])
            if j == nOP_y-1:
                if guide_phase > np.pi:
                    guide_phase = 2*np.pi - guide_phase
                if guide_phase < 0:
                    guide_phase = -guide_phase
                guides.append(guide_phase)
                ax.text(center[0]-per_dis[i]/4, center[1] + per_dis[i]/2*1.5, f'{guide_phase*180/np.pi:0.2f}', fontsize = 6)
            ax.plot(guide_x, guide_y, ':k', lw = 0.5)
            
            for l in range(nblock[i]):
                if np.linalg.norm(block_center[i][l,:]-center) < block_r+r*np.sqrt(2):                    
                    OP_set, pick = square_OP(center, r, phase0, phase, pos[i][l,:,:], np.mod(j+k,2)) #np.mod(i + j,2)
                    OP[i][l, pick] = (OP_set[pick] + np.pi/2)/np.pi
                    if get_condition(i,j,k) and check:
                        OP_row += OP[i][l, pick].tolist()
                        
                    if plot and get_condition(i,j,k):
                        npick = np.sum(pick)
                        hsv = np.zeros((npick,3))
                        hsv[:,1] = sat0
                        hsv[:,2] = val0
                        hue = OP[i][l,pick]
                        hsv[:,0] = hue
                        ax.scatter(pos[i][l,0,pick], pos[i][l,1,pick], s = 0.25, marker = marker[i], color = clr.hsv_to_rgb(hsv), edgecolors = 'none')
                        ax.plot(center[0],center[1], marker = marker[i], color = 'k', ms = 0.5)
                        
    ax.plot(block_center[i][:,0], block_center[i][:,1], 'xk', ms = 0.5)
    for l in range(nblock[i]):
        ax.plot(pos[i][l,0,:], pos[i][l,1,:], ',k')
print(guides[1]*180/np.pi - 45, (guides[-1] - guides[0])*180/np.pi)
if check:
    check_ax = check_fig.add_subplot(111)
    check_ax.hist(np.array(OP_row)*180, bins = 48)
if plot:
    ax.set_xlabel('cortical-x (mm)',fontsize=7)
    ax.set_ylabel('cortical-y (mm)',fontsize=7)
    ax.tick_params(axis = 'x', labelsize = 6)
    ax.tick_params(axis = 'y', labelsize = 6)
    ax.set_aspect('equal')
    fig.savefig('pinwheel.png')

In [ ]:
feature_filename = fdr+'V1_feature'+theme+'.bin'
with open(feature_filename, 'wb') as f:
    nFeature = 2
    np.array([nFeature], dtype = 'u4').tofile(f)
    ## feature1: OD
    #nfl = 3 # V1 layers that have this feature, 0: L2/3, 1:L4-beta, 2:L4-alpha
    #np.array([nfl, 0, 1, 2], dtype = 'u4').tofile(f)
    #nV1.astype('u4').tofile(f)
    LR[0].astype('f4').tofile(f)
    #LR[1].astype('f4').tofile(f)
    #LR[2].astype('f4').tofile(f)
    ## feature2: OP
    #nfl = 3 # V1 layers that have this feature, 0: L2/3, 1:L4-beta, 2:L4-alpha
    #np.array([nfl, 0, 1, 2], dtype = 'u4').tofile(f) 
    #nV1.astype('u4').tofile(f)
    OP[0].astype('f4').tofile(f)
    #OP[1].astype('f4').tofile(f)
    #OP[2].astype('f4').tofile(f)
#for i in range(3):
#    print(np.min(LR[i]), np.max(LR[i]))
#    print(np.min(OP[i]), np.max(OP[i]))

In [ ]:
fig = plt.figure(figsize = (4,2), dpi = 120)

#nbin_list = np.array([6,12,24,48])
nbin_list = np.array([90])
for i in range(nbin_list.size):
    nbins = nbin_list[i]
    ax = fig.add_subplot(nbin_list.size , 1, i+1)
    ax.hist(OP[0].flatten() * 180, bins = nbins)

In [ ]:
# Excluding S_on, S_off from LGN Koniocellular layers to L1, L2/3 for now

ratio = 8/np.pi # from half a circle to square
# Malpeli et al 1996
parvo_den_alt = lambda E: 1011688*(E+2.9144)**(-2.6798) # cells/deg^2
parvo_slice = lambda E: np.pi*E*parvo_den_alt(E)
result = integrate.quad(parvo_slice, 0, ecc)
nparvo = round(result[0]*ratio)
magno_den_alt = lambda E: 2620.2*((E-1.8322)**2+5.5638)**(-0.8012) # cells/deg^2
magno_slice = lambda E: np.pi*E*magno_den_alt(E)
result = integrate.quad(magno_slice, 0, ecc)
nmagno = round(result[0]*ratio)
nparvo = ((nparvo + warpSize-1)//warpSize)*warpSize
nmagno = ((nmagno + warpSize-1)//warpSize)*warpSize

# C-On:Off ~ 3:1; (Wiesel and Hubel 1966) cone-to-ganglion ~ 1:1 (McMahon et al., 2000)
# Red:Green (L:M), 1.5-2.0 overall, very large variations(Deeb et al., 2000, Bowmaker et. al., 2003)
rL_on = 0 # 6
rL_off = 1 # 2
rM_on = 2 # 3
rM_off = 3 # 1
nparvo_warp = nparvo//warpSize
nmagno_warp = nmagno//warpSize
nmagno_warp = int(round(nmagno_warp/2)) * 2
nparvo_warp = int(round(nparvo_warp/12)) * 12
nparvo = nparvo_warp * warpSize
nmagno = nmagno_warp * warpSize
print(f'{nparvo} parvo cells, {nmagno} cells')
RG_OnOff = np.zeros(nparvo)
i = 0
RG_OnOff[i:i+nparvo//12*3] = rL_on
#i += nparvo//12*6
i += nparvo//12*3
RG_OnOff[i:i+nparvo//12*3] = rL_off
#i += nparvo//12*2
i += nparvo//12*3
RG_OnOff[i:i+nparvo//12*3] = rM_on
#i += nparvo//12*3
i += nparvo//12*3
RG_OnOff[i:i+nparvo//12*3] = rM_off
#i += nparvo//12*1
i += nparvo//12*3
assert(i==nparvo)
OnOff = np.zeros(nmagno)
i = 0
OnOff[i:i+nmagno//2] = 4
i += nmagno//2
OnOff[i:i+nmagno//2] = 5
i += nmagno//2
assert(i==nmagno)

In [ ]:
parvo_cart = square_pos(ecc*2, nparvo, np.zeros(2))
xyID, w = assign_square_xyID(nparvo, (parvo_cart + ecc)/(2*ecc))

In [ ]:
fig = plt.figure(figsize = (4,4), dpi = 300)
ax = fig.add_subplot(111)
parvo_mc = [',r', ',m', ',g', ',c']
for i in range(4):
    pick = RG_OnOff == i
    ax.plot(parvo_cart[0,pick], parvo_cart[1,pick], parvo_mc[i])
        
check_list = np.array([[2, 2, w-2, w-2], [2, w-2, 2, w-2]])
for i in range(4):
    check = xyID - check_list[:,i].reshape(2,1) == 0
    idx = np.nonzero(check.all(0))[0]
    if idx.size > 0:
        j = idx[0]
        ax.text(parvo_cart[0,j], parvo_cart[1,j], f'{xyID[0,j]}, {xyID[1,j]}', fontsize = 5)
        if idx.size > 1:
            raise Exception('overlap id')
    
ax.set_aspect('equal')
ax.tick_params(axis = 'x', labelsize = 7)
ax.tick_params(axis = 'y', labelsize = 7)
print(w)

In [ ]:
LGN_surfaceID_file = fdr + 'LGN_surfaceID' + theme + '.bin'
with open(LGN_surfaceID_file, 'wb') as f:
    np.array([w, w]).astype('u4').tofile(f)
    xyID.tofile(f)

In [ ]:
def cart2polar(x, y):
    theta = np.arctan2(y, x)
    r = np.sqrt(x*x + y*y)
    return theta, r
parvo_polar = np.empty(parvo_cart.shape)
parvo_polar[0,:], parvo_polar[1,:] = cart2polar(parvo_cart[0,:], parvo_cart[1,:])

In [ ]:
LGN_vpos_file = fdr + 'LGN_vpos' + theme + '.bin'
with open(LGN_vpos_file, 'wb') as f:
    #       #type,  # ipsi-     | # contra-
    np.array([nparvo, 0], dtype = 'u4').tofile(f)
    np.array([ecc], dtype = fp).tofile(f)
    # cart
    #         xmin, xmax, ymin, ymax
    np.array([-ecc, 2*ecc, -ecc, 2*ecc], dtype = fp).tofile(f)
    parvo_cart.astype(fp).tofile(f)
    # type
    RG_OnOff.astype('u4').tofile(f)
    # polar
    parvo_polar.astype(fp).tofile(f)